In [1]:
import pandas as pd
import os
import gc

print("loading data ...")
hack = pd.read_csv("small_player_starts_train.csv")

loading data ...


In [2]:
print("Amount of events: ", len(hack))
users = hack.user_id.unique()
print("Unique users: ", len(users))
print("Unique items: ", hack.item_id.nunique())
print("Min date: ", hack.date.min())
print("Max date: ", hack.date.max())

Amount of events:  2439644
Unique users:  1313891
Unique items:  261288
Min date:  2023-08-21 00:00:00+03:00
Max date:  2023-08-21 23:59:59+03:00


In [3]:
hack['date_short'] = hack.date.apply(lambda l: l.split(" ")[0])

In [4]:
%%time
groupby = hack.groupby(["user_id", "date_short"]).count()
users_active = groupby[groupby['item_id'] >= 2].index.unique()

CPU times: total: 2.34 s
Wall time: 2.35 s


In [5]:
users_active = set(users_active)
users_active

{('user_25725793', '2023-08-21'),
 ('user_24781525', '2023-08-21'),
 ('user_17370900', '2023-08-21'),
 ('user_13506586', '2023-08-21'),
 ('user_4005252', '2023-08-21'),
 ('user_3330949', '2023-08-21'),
 ('user_9245607', '2023-08-21'),
 ('user_2315649', '2023-08-21'),
 ('user_678515', '2023-08-21'),
 ('user_2743737', '2023-08-21'),
 ('user_25398886', '2023-08-21'),
 ('user_4512196', '2023-08-21'),
 ('user_9541635', '2023-08-21'),
 ('user_9136553', '2023-08-21'),
 ('user_21616852', '2023-08-21'),
 ('user_9563616', '2023-08-21'),
 ('user_17526332', '2023-08-21'),
 ('user_1755328', '2023-08-21'),
 ('user_29115650', '2023-08-21'),
 ('user_24320055', '2023-08-21'),
 ('user_18808884', '2023-08-21'),
 ('user_26313802', '2023-08-21'),
 ('user_24489890', '2023-08-21'),
 ('user_3857651', '2023-08-21'),
 ('user_25940031', '2023-08-21'),
 ('user_20547013', '2023-08-21'),
 ('user_26934749', '2023-08-21'),
 ('user_14166821', '2023-08-21'),
 ('user_18319409', '2023-08-21'),
 ('user_7044087', '2023-08-

In [7]:
from tqdm import tqdm 
tqdm.pandas()
hack['active'] = hack.progress_apply(lambda l: True if (l.user_id, l.date_short) in users_active else False, axis=1)
hack=hack[hack.active].reset_index()
hack

100%|██████████| 2439644/2439644 [00:18<00:00, 128819.66it/s]


,index,date,user_id,item_id,watch_time,is_autorized,date_short,active
0,0,2023-08-21 15:53:30+03:00,user_7941459,video_1535803,22,0,2023-08-21,True
1,2,2023-08-21 21:18:12+03:00,user_25219604,video_1905049,11,0,2023-08-21,True
2,4,2023-08-21 17:24:32+03:00,user_25483180,video_500499,20,0,2023-08-21,True
3,6,2023-08-21 15:21:19+03:00,user_22148320,video_1366097,32,0,2023-08-21,True
4,9,2023-08-21 00:35:41+03:00,user_14452025,video_283933,2142,0,2023-08-21,True
...,...,...,...,...,...,...,...,...
1457171,2439638,2023-08-21 16:44:47+03:00,user_11895299,video_853811,101,0,2023-08-21,True
1457172,2439639,2023-08-21 02:51:53+03:00,user_15478739,video_1449287,291,1,2023-08-21,True
1457173,2439641,2023-08-21 05:19:55+03:00,user_3507470,video_464555,261,0,2023-08-21,True
1457174,2439642,2023-08-21 12:32:32+03:00,user_13128840,video_420973,21,0,2023-08-21,True


In [8]:
print("Финальное число событий", len(hack))
print("Финальное число видео", hack.item_id.nunique())

Финальное число событий 1457176
Финальное число видео 164056


In [12]:
features = pd.read_parquet("videos.parquet")

FileNotFoundError: [Errno 2] No such file or directory: 'videos.parquet'

In [13]:
hack = hack[hack.watch_time >= 30]
un_items = hack['item_id'].unique()
features = features[features['item_id'].isin(un_items)]
features = features.sort_values(by=['item_id']).reset_index()
features


NameError: name 'features' is not defined

In [11]:
group_item = hack.groupby('item_id').agg(watch_time=('watch_time',list), ind=('index', list))
group_item['duration'] = features['duration'].to_list()

NameError: name 'features' is not defined

In [ ]:
group_item[['watch_time', 'duration']]

In [ ]:
true_index = []
for id, row in group_item.iterrows():
    for i in range(len(row.watch_time)):
        if row.duration <= 300000 or row.duration > 300000 and row.watch_time[i] * 1000 > row.duration * 0.25:
            true_index.append(row.ind[i])

In [ ]:
group_item.loc[['video_10000', 'video_1000000']].duration.to_list()

In [ ]:
hack = hack[hack['index'].isin(true_index)]

In [ ]:
len(hack)

In [ ]:
def split_data(data: pd.DataFrame, group_item, step=1):
    users_df = hack.groupby('user_id').agg(item_id=('item_id', list), watch_time=('watch_time',list))
    users_df = users_df[users_df['item_id'].map(len) > 1]
    train_duration = []
    train_watch_time = []
    test_lst, train_lst = [], []
    for id, row in users_df.iterrows():
        train_lst.append(row.item_id[:-1])
        train_watch_time.append(row.watch_time[:-1])
        test_lst.append(row.item_id[-1])
        train_duration.append(group_item.loc[row.item_id[:-1]].duration.to_list())
    test_df = pd.DataFrame({'user_id': users_df.index.to_list(), 'item_id': test_lst})
    train_df = pd.DataFrame({'user_id': users_df.index.to_list(), 'item_id': train_lst, 'watch_time': train_watch_time, 'duration': train_duration})
    return test_df, train_df

In [ ]:
test_df, train_df = split_data(hack, group_item)

In [ ]:
train_df.to_csv(index=False)

In [ ]:
hack.to_csv("hack", index=False)

In [ ]:
train_df.to_csv("train", index=False)

In [ ]:
test_df

In [ ]:
print(type(users_df))

In [ ]:
import numpy
import sklearn
from sklearn import metrics

pred_scores = [0.7, 0.3, 0.5, 0.6, 0.55, 0.9, 0.4, 0.2, 0.4, 0.3]
y_true = ["positive", "negative", "negative", "positive", "positive", "positive", "negative", "positive", "negative", "positive"]

threshold = 0.5
y_pred = ["positive" if score >= threshold else "negative" for score in pred_scores]
print(y_pred)

In [ ]:
r = numpy.flip(metrics.confusion_matrix(y_true, y_pred))
print(r)

precision = metrics.precision_score(y_true=y_true, y_pred=y_pred, pos_label="positive")
print(precision)

recall = metrics.recall_score(y_true=y_true, y_pred=y_pred, pos_label="positive")
print(recall)